'votakvot' basic usage example

In [ ]:
import math
import random
import tempfile

In [ ]:
import votakvot

In [ ]:
store_path = tempfile.mkdtemp()
print("write results into", store_path)

In [ ]:
votakvot.init(
    
    # path to store results
    path=store_path,   
    
    # execute code in separate processes
    # add this if you want to use parallel execution
    runner='process',  
)

In [ ]:
@votakvot.track()
def calc_pi(n, seed=0):
    
    r = random.Random(seed)
    acc = 0
    for _i in range(n):
        x = r.random()
        y = r.random()
        acc += x * x + y * y < 1

    pi = 4 * (acc / n)

    votakvot.inform(
        accuracy=abs(math.pi - pi),
    )

    return  pi

In [ ]:
# single call

calc_pi(100, 0)

In [ ]:
# sequential run

for s in range(30):
    n = (10 + s) * 10 ** 5
    pi = calc_pi(n, s)
    print(f"n={n}, s={s}  >>  pi={pi}")

In [ ]:
# parallel run
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=10)

results = executor.map(
    lambda params: {**params, 'pi': calc_pi(**params)},
    [{'n': (10 + s) * 10 ** 5, 'seed': s} for s in range(30)]
)
for r in results:
    print(">>", r)

In [ ]:
# load results

votakvot.load_report()